# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/eugeneklyashtorny/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/eugeneklyashtorny/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [3]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [4]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [5]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [6]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [7]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [8]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node 'ee593d'. Skipping!
Property 'summary' already exists in node '7aaa90'. Skipping!
Property 'summary' already exists in node '68f0fa'. Skipping!
Property 'summary' already exists in node 'fb4c62'. Skipping!
Property 'summary' already exists in node '23a3e8'. Skipping!
Property 'summary' already exists in node '648ad9'. Skipping!
Property 'summary' already exists in node '534228'. Skipping!
Property 'summary' already exists in node '77783f'. Skipping!
Property 'summary' already exists in node '03f2d0'. Skipping!
Property 'summary' already exists in node '8793af'. Skipping!
Property 'summary' already exists in node '73a33c'. Skipping!
Property 'summary' already exists in node '605b16'. Skipping!
Property 'summary' already exists in node 'b80b4c'. Skipping!
Property 'summary' already exists in node '7eeaa9'. Skipping!


Applying CustomNodeFilter:   0%|          | 0/6 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/43 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node 'ee593d'. Skipping!
Property 'summary_embedding' already exists in node '68f0fa'. Skipping!
Property 'summary_embedding' already exists in node '77783f'. Skipping!
Property 'summary_embedding' already exists in node '648ad9'. Skipping!
Property 'summary_embedding' already exists in node '23a3e8'. Skipping!
Property 'summary_embedding' already exists in node 'b80b4c'. Skipping!
Property 'summary_embedding' already exists in node '7aaa90'. Skipping!
Property 'summary_embedding' already exists in node '534228'. Skipping!
Property 'summary_embedding' already exists in node 'fb4c62'. Skipping!
Property 'summary_embedding' already exists in node '8793af'. Skipping!
Property 'summary_embedding' already exists in node '73a33c'. Skipping!
Property 'summary_embedding' already exists in node '03f2d0'. Skipping!
Property 'summary_embedding' already exists in node '605b16'. Skipping!
Property 'summary_embedding' already exists in node '7eeaa9'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 40, relationships: 478)

We can save and load our knowledge graphs as follows.

In [9]:
kg.save("loan_data_kg.json")
loan_data_kg = KnowledgeGraph.load("loan_data_kg.json")
loan_data_kg

KnowledgeGraph(nodes: 40, relationships: 478)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [10]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loan_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [11]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


| Synthesizer                             | What it does (plain‑English)                                                                                                                                                                                                                                                 | Typical use‑case                                                                                                                               |
| --------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------------------------- |
| **AbstractQuerySynthesizer**            | Looks at a *cluster* of related document chunks, invents a short **theme/summary** for the cluster, and then writes broad, conceptual questions about that theme. Think of it as asking “big‑picture” questions whose answers may be scattered across the docs. ([Ragas][1]) | Good for testing whether your RAG pipeline can retrieve and answer open‑ended or thematic questions.                                           |
| **ComparativeAbstractQuerySynthesizer** | First finds a **common concept** shared across multiple chunks/key‑phrases, then creates questions that explicitly **compare or contrast** those pieces of information (e.g., “How does X differ from Y?”). It stresses multi‑chunk reasoning. ([Ragas][1])                  | Helps you see if the retriever can pull together information from different parts of your knowledge base and support comparison‑style answers. |
| **SpecificQuerySynthesizer**            | Chooses one **precise chunk & key‑phrase**, then generates a direct, fact‑based question about that snippet. These are the “needle‑in‑the‑haystack” queries that expect a single, concrete answer. ([Ragas][1])                                                              | Useful for measuring how accurately the system surfaces exact facts or definitions buried in the corpus.                                       |

[1]: https://docs.ragas.io/en/v0.2.0/references/synthesizers/ "Synthesizers - Ragas"


Finally, we can use our `TestSetGenerator` to generate our testset!

In [12]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,School Participation Division what do?,"[Chapter 1 Academic Years, Academic Calendars,...",The context does not provide specific details ...,single_hop_specifc_query_synthesizer
1,What is 34 CFR 668.3(a)?,[Regulatory Citations Academic year minimums: ...,Regulatory Citations Academic year minimums: 3...,single_hop_specifc_query_synthesizer
2,How does Volume 8 address the inclusion of cli...,[Inclusion of Clinical Work in a Standard Term...,Inclusion of clinical work in a standard term ...,single_hop_specifc_query_synthesizer
3,Can you explain what the term 'Title IV' refer...,[Non-Term Characteristics A program that measu...,Payment periods are applicable to all Title IV...,single_hop_specifc_query_synthesizer
4,How does the TEACH Grrant affect disbursement ...,[both the credit or clock hours and the weeks ...,The TEACH Grant amount that a student is eligi...,single_hop_specifc_query_synthesizer
5,How does the inclusion of clinical work in sta...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The inclusion of clinical work in standard ter...,multi_hop_abstract_query_synthesizer
6,How does the variation in defining academic ye...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The context explains that academic years must ...,multi_hop_abstract_query_synthesizer
7,How do the different academic year definitions...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The context explains that each eligible progra...,multi_hop_abstract_query_synthesizer
8,"In Appendix A and B, how do disbursement rules...",[<1-hop>\n\nboth the credit or clock hours and...,"According to the context, Appendix A explains ...",multi_hop_specific_query_synthesizer
9,which volume 2 or 7 is more important for disb...,[<1-hop>\n\nboth the credit or clock hours and...,The context indicates that Volume 2 discusses ...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [13]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '71741c'. Skipping!
Property 'summary' already exists in node 'd546b3'. Skipping!
Property 'summary' already exists in node '052551'. Skipping!
Property 'summary' already exists in node '9c8934'. Skipping!
Property 'summary' already exists in node '3db8e0'. Skipping!
Property 'summary' already exists in node '01fe9a'. Skipping!
Property 'summary' already exists in node '0c15dd'. Skipping!
Property 'summary' already exists in node '537c01'. Skipping!
Property 'summary' already exists in node 'c9586b'. Skipping!
Property 'summary' already exists in node 'c94180'. Skipping!
Property 'summary' already exists in node '29c239'. Skipping!
Property 'summary' already exists in node 'e05f30'. Skipping!
Property 'summary' already exists in node 'b48518'. Skipping!
Property 'summary' already exists in node 'f5a262'. Skipping!


...

Property 'summary_embedding' already exists in node '29c239'. Skipping!
Property 'summary_embedding' already exists in node 'd546b3'. Skipping!
Property 'summary_embedding' already exists in node 'c94180'. Skipping!
Property 'summary_embedding' already exists in node '3db8e0'. Skipping!
Property 'summary_embedding' already exists in node '01fe9a'. Skipping!
Property 'summary_embedding' already exists in node '71741c'. Skipping!
Property 'summary_embedding' already exists in node '052551'. Skipping!
Property 'summary_embedding' already exists in node 'c9586b'. Skipping!
Property 'summary_embedding' already exists in node 'f5a262'. Skipping!
Property 'summary_embedding' already exists in node '537c01'. Skipping!
Property 'summary_embedding' already exists in node 'b48518'. Skipping!
Property 'summary_embedding' already exists in node '9c8934'. Skipping!
Property 'summary_embedding' already exists in node '0c15dd'. Skipping!
Property 'summary_embedding' already exists in node 'e05f30'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [14]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,Can you explain what the Knowledge Center is a...,"[Chapter 1 Academic Years, Academic Calendars,...",The context provided does not explicitly defin...,single_hop_specifc_query_synthesizer
1,What does 34 CFR 668.3(b) specify?,[Regulatory Citations Academic year minimums: ...,34 CFR 668.3(b) specifies the weeks of instruc...,single_hop_specifc_query_synthesizer
2,Chapter 3 include clinical work?,[Inclusion of Clinical Work in a Standard Term...,Inclusion of clinical work in a standard term ...,single_hop_specifc_query_synthesizer
3,Is the Federal Work-Study program subject to p...,[Non-Term Characteristics A program that measu...,The payment period is applicable to all Title ...,single_hop_specifc_query_synthesizer
4,Wht conditions for clinical work inclusion in ...,[<1-hop>\n\nInclusion of Clinical Work in a St...,Clinical work can be included in a standard te...,multi_hop_abstract_query_synthesizer
5,How do clock-hour and non-term credit-hour pro...,[<1-hop>\n\nboth the credit or clock hours and...,In clock-hour and non-term credit-hour program...,multi_hop_abstract_query_synthesizer
6,How do nonstandard terms and sequential course...,[<1-hop>\n\nInclusion of Clinical Work in a St...,"Nonstandard terms, such as those shorter than ...",multi_hop_abstract_query_synthesizer
7,How do the different academic year definitions...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The context explains that each eligible progra...,multi_hop_abstract_query_synthesizer
8,Volume 2 and Volume 7 info about disbursement ...,[<1-hop>\n\nboth the credit or clock hours and...,Volume 2 explains that disbursement of federal...,multi_hop_specific_query_synthesizer
9,whre is appendix A and B in disbursement rules?,[<1-hop>\n\nDisbursement Timing in Subscriptio...,Appendix A and Appendix B are referenced in th...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [15]:
from langsmith import Client

client = Client()

dataset_name = "Loan Synthetic Data"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Loan Synthetic Data"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [16]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [17]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [19]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [20]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan RAG"
)

In [21]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [22]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [23]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [24]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [25]:
rag_chain.invoke({"question" : "What kinds of loans are available?"})

'The kinds of loans available mentioned in the context include:\n\n- Direct Subsidized Loans (available only to undergraduate students)\n- Direct Unsubsidized Loans\n- Direct PLUS Loans (student Federal PLUS Loan and parent Direct PLUS Loan on behalf of a dependent student)\n- Federal Stafford Loans (Subsidized and Unsubsidized, made under the FFEL Program before July 1, 2010)\n- Federal SLS Loans\n- Federal PLUS Loans (made under the FFEL Program before July 1, 2010)\n- Direct Consolidation Loans\n- Federal Consolidation Loans (made under the FFEL Program)\n\nNote: No new FFEL Program loans have been made since July 1, 2010.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [26]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [27]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `empathy_evaluator`:

| Evaluator                       | What it is scoring                                                                                                             | How it decides                                                                                                                                                        |
| ------------------------------- | ------------------------------------------------------------------------------------------------------------------------------ | --------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `qa_evaluator`                  | **Answer correctness & completeness** — Does the chain’s answer actually address the question and match the reference answer?  | Uses the built‑in `"qa"` rubric, comparing `prediction` vs. `reference` for factual overlap, relevance, and sufficiency.                                              |
| `labeled_helpfulness_evaluator` | **Helpfulness (label‑aware)** — Given both the user question and the ground‑truth answer, how helpful is the model’s response? | Uses the `"labeled_criteria"` mode with a single criterion `helpfulness`, so the LLM judges whether the response *adds value for the user* relative to the reference. |
| `empathy_evaluator`             | **Empathy / tone** — Does the response make the user feel heard and understood?                                                | Uses the generic `"criteria"` evaluator with one custom criterion `empathy`; it ignores the reference answer and focuses purely on the response’s emotional tone.     |


## LangSmith Evaluation

In [28]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'crazy-mitten-91' at:
https://smith.langchain.com/o/a8b64252-5f0f-4f35-a048-c004586e098a/datasets/81e1785c-c4e7-435d-9bf1-58fe832cdf8f/compare?selectedSessions=3b66adf8-1c0a-4d2e-914d-07c1733c4e7c




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How do Chapters 2 and 3 collectively explain t...,I don't know.,None,Chapter 2 details the requirements for definin...,0,0,0,2.425654,fe9e50c3-a947-434e-86bf-36624dac5344,2c20d08e-14d9-4c21-9110-905780d5e517
1,How do Volume 8 and Volume 7 relate to the tim...,"Based on the provided context, Volume 7 and Vo...",None,"According to the provided context, Volume 8 di...",1,1,0,9.169424,b9057a0d-3b7a-4635-93a6-c115ed9d9ca6,1a07f4f2-ff87-4069-9513-cd6def86089b
2,whre is appendix A and B in disbursement rules?,"Based on the provided context, Appendix B is m...",None,Appendix A and Appendix B are referenced in th...,0,0,0,3.006635,16df9b8b-e081-4335-8850-08de7668107f,8071ee67-4783-46c4-9b1a-0f78ecf1ecac
3,Volume 2 and Volume 7 info about disbursement ...,"Based on the provided context, information rel...",None,Volume 2 explains that disbursement of federal...,0,0,0,17.175460,90d3886f-bee4-4936-b8ae-7a59a6b7f75f,128b6242-1a75-4b3d-a3a5-edfd3b079a6d
4,How do the different academic year definitions...,The regulations require that for both undergra...,None,The context explains that each eligible progra...,1,1,0,5.855138,11b81f85-d81a-4d4c-9726-4c7731e0bbc0,132800db-4ea0-453d-b7eb-1a7613850b7f
5,How do nonstandard terms and sequential course...,"Based on the provided context, nonstandard ter...",None,"Nonstandard terms, such as those shorter than ...",1,1,0,6.202939,9f86007b-3618-45d5-999b-c0d75b8be7f6,a045de01-bc4b-4357-adb0-d9fce9b9a9be
6,How do clock-hour and non-term credit-hour pro...,Based on the provided context:\n\nFor clock-ho...,None,In clock-hour and non-term credit-hour program...,1,0,0,15.905004,8c232290-b229-4625-baf8-24d185efd1c1,e359054a-b09e-4efb-bf73-84eea70ffc32
7,Wht conditions for clinical work inclusion in ...,Based on the provided context:\n\n**Conditions...,None,Clinical work can be included in a standard te...,1,1,0,6.821733,fd75f318-755b-4f41-a89f-6d37c6d3dd2a,216b031d-734e-4c2d-99c1-a33cb0d89338
8,Is the Federal Work-Study program subject to p...,"No, the Federal Work-Study (FWS) program is no...",None,The payment period is applicable to all Title ...,1,1,0,2.466674,c5762f9f-6a60-4738-a0aa-45c5892ee6a8,a94d7273-aff7-417b-88b6-df672a83fffc
9,Chapter 3 include clinical work?,"Based on the provided context, Chapter 3 inclu...",None,Inclusion of clinical work in a standard term ...,1,1,0,3.984946,f3583fb1-05ab-4b25-9a65-52c7f678a021,ab095fdc-4b20-4eb8-8ca5-3b86244f8531


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [44]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [45]:
rag_documents = docs

In [46]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

| Design choice                               | What changes when you **shrink** chunks (fewer tokens each)                                                                                                                                        | What changes when you **grow** chunks (more tokens each)                                                                                                        |
| ------------------------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Semantic signal per embedding**           | • Embedding represents a *single, narrow idea* → higher **precision** for pinpoint questions.<br>• Risk of losing surrounding context, so answer may need several chunks retrieved instead of one. | • Embedding mixes multiple ideas → higher **contextual coherence**, but can become “blurred” and hurt similarity scoring for very specific queries.             |
| **Recall vs. precision**                    | • More, finer‑grained vectors → better **recall** (the exact fact is likely in at least one chunk).<br>• More results returned = more noise to rank/filter.                                        | • Fewer vectors → lower recall for needle‑in‑haystack facts, but each hit already contains lots of context (higher **precision**).                              |
| **Context window budget**                   | • Need to concatenate **more** retrieved chunks to give the model enough context → can overflow token window or crowd out prompt/system tokens.                                                    | • Often one large chunk is enough; fewer chunks pasted into the prompt saves tokens.                                                                            |
| **Latency & cost**                          | • More chunks stored → larger vector DB, more similarity calculations, higher latency and \$\$ for retrieval & embeddings.                                                                         | • Fewer, bigger chunks → smaller index & faster lookup, cheaper embedding bill, but each fetch brings more tokens to the LLM (slightly higher generation cost). |
| **Risk of hallucination / irrelevant info** | • Model may receive only the sliver containing the answer → less irrelevant text, lower hallucination risk.                                                                                        | • Chunk may include unrelated sentences → LLM can pick up spurious text and hallucinate or give verbose, off‑topic answers.                                     |
| **Evaluation metrics you’ll see move**      | • ↑ hit‑rate @ 1, but possibly ↓ exact‑match F1 if the retriever fails to pull *all* needed fragments.<br>• Longer generation latency (more retrieved docs).                                       | • ↑ answer completeness, but ↓ hit‑rate for ultra‑specific eval queries.<br>• Smaller retrieval latency; generation may be token‑heavier.                       |


In [47]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

| What can change                             | **Why the embedding model matters**                                                                                                                                                                                                                                                                                                                                                                                     | Impact you’ll notice                                                                                                                                                   |
| ------------------------------------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Retrieval accuracy (precision & recall)** | Different models learn different semantic spaces. A stronger model (e.g., OpenAI’s *text‑embedding‑3‑large* or Google’s *gemini‑embedding‑001*) captures finer‑grained meaning and paraphrases, so similar passages land closer together in vector space. In benchmarks like MIRACL and MTEB these newer models boost retrieval scores by **20‑40 pp** over older baselines. ([OpenAI][1], [Google Developers Blog][2]) | ‑ Fewer “misses” on exact‑fact queries.<br>‑ Better ranking of truly relevant chunks, which lifts end‑to‑end QA metrics (Exact‑Match, F1, RAGAS “answer correctness”). |
| **Domain fit & task alignment**             | Some models are **general‑purpose**; others are tuned for code, biomedicine, or emotion. A domain‑specialized encoder embeds your corpus with terminology the model already “understands,” so cosine similarity is more reliable. ([DataStax][3])                                                                                                                                                                       | ‑ Higher hit‑rate for jargon‑heavy queries.<br>‑ Less need for prompt‑engineering hacks or extra retriever filters.                                                    |
| **Multilingual coverage**                   | Multilingual encoders (e.g., *gemini‑embedding‑001*) learn aligned spaces across >100 languages. If users ask in Spanish but docs are in English, a multilingual model can still retrieve correctly. ([Google Developers Blog][2])                                                                                                                                                                                      | ‑ Cross‑lingual Q\&A works “out of the box.”<br>‑ No separate index per language, lowering infra complexity.                                                           |
| **Vector dimensionality & index size**      | Models output 256‑ to 3 072‑dim vectors. Higher dimensions preserve nuance but grow index RAM/disk and slow similarity search unless you use more aggressive ANN parameters. ([OpenAI][1])                                                                                                                                                                                                                              | ‑ Trade‑off between retrieval speed/cost and quality.<br>‑ May force you to tweak HNSW/IVF settings or shard the DB.                                                   |
| **Inference latency & \$\$**                | Larger or server‑side models cost more per 1 K tokens and add round‑trip latency. A smaller on‑prem model (e.g., MiniLM) is cheaper but may drop accuracy.                                                                                                                                                                                                                                                              | ‑ Direct effect on user wait‑time and cloud bill.<br>‑ Could influence how many chunks you can embed within a budget.                                                  |
| **Licensing & data‑governance constraints** | Some best‑in‑class encoders are **closed** (OpenAI, Google). Open‑source options on Hugging Face (Stella, bge‑large) offer self‑hosting but different quality/IP terms. ([DataStax][3])                                                                                                                                                                                                                                 | ‑ Determines whether you can run fully on‑prem.<br>‑ Impacts compliance for sensitive data.                                                                            |

[1]: https://openai.com/index/new-embedding-models-and-api-updates/?utm_source=chatgpt.com "New embedding models and API updates - OpenAI"
[2]: https://developers.googleblog.com/en/gemini-embedding-available-gemini-api/?utm_source=chatgpt.com "Gemini Embedding now generally available in the Gemini API"
[3]: https://www.datastax.com/blog/best-embedding-models-information-retrieval-2025?utm_source=chatgpt.com "The Best Embedding Models for Information Retrieval in 2025"


In [ ]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan Data for RAG"
)

In [ ]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [ ]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [ ]:
empathy_rag_chain.invoke({"question" : "What kinds of loans are available?"})

"Thank you for your question! Based on the information provided in the context, there are several types of federal student loans available:\n\n1. **Direct Subsidized Loans** – These loans are available to students with financial need, and the government pays the interest while the student is in school at least half-time.\n\n2. **Direct Unsubsidized Loans** – These loans are available to students regardless of financial need, and interest accrues while the student is in school.\n\n3. **Direct PLUS Loans** – These loans are for parents of dependent students (or for graduate/professional students themselves) who may borrow up to the cost of attendance minus other financial aid. Parents must meet eligibility requirements, and these loans do not have a fixed limit but cannot exceed the student's cost of attendance.\n\nAlso, there are mentions of loan combinations, where students may qualify for both Direct Subsidized and Direct Unsubsidized Loans, and additional Direct Unsubsidized amounts 

Finally, we can evaluate the new chain on the same test set!

In [ ]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'crazy-ornament-60' at:
https://smith.langchain.com/o/a8b64252-5f0f-4f35-a048-c004586e098a/datasets/81e1785c-c4e7-435d-9bf1-58fe832cdf8f/compare?selectedSessions=74ffa5cf-0357-4258-b1ae-5dcc25503e02




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How do Chapters 2 and 3 collectively explain t...,Thank you for your thoughtful question. From t...,None,Chapter 2 details the requirements for definin...,1,0,1,6.594259,fe9e50c3-a947-434e-86bf-36624dac5344,3dbd3e6c-1f56-4df8-bb17-fb47246ca45a
1,How do Volume 8 and Volume 7 relate to the tim...,Thank you for your thoughtful question. It sho...,None,"According to the provided context, Volume 8 di...",1,1,1,6.514898,b9057a0d-3b7a-4635-93a6-c115ed9d9ca6,d36ae6dc-f556-479c-a2bb-77b29c1d7e85
2,whre is appendix A and B in disbursement rules?,Thank you for your question—it's really import...,None,Appendix A and Appendix B are referenced in th...,0,0,1,6.299075,16df9b8b-e081-4335-8850-08de7668107f,86d90a44-3869-4751-83f0-14743e7a8507
3,Volume 2 and Volume 7 info about disbursement ...,Thank you for your question—it's really import...,None,Volume 2 explains that disbursement of federal...,0,0,1,8.585288,90d3886f-bee4-4936-b8ae-7a59a6b7f75f,3e7c9170-fe20-4f62-9b39-3c0d552b5158
4,How do the different academic year definitions...,Thank you for your thoughtful question. Based ...,None,The context explains that each eligible progra...,1,1,1,5.421423,11b81f85-d81a-4d4c-9726-4c7731e0bbc0,fddf5e82-80c3-4e5e-8ea8-16f66f09b3fe
5,How do nonstandard terms and sequential course...,Thank you for your thoughtful question. Based ...,None,"Nonstandard terms, such as those shorter than ...",1,1,1,8.103996,9f86007b-3618-45d5-999b-c0d75b8be7f6,f5a7707a-8955-487c-b97f-ec60799841fd
6,How do clock-hour and non-term credit-hour pro...,I can see that you're trying to understand som...,None,In clock-hour and non-term credit-hour program...,0,0,1,8.473731,8c232290-b229-4625-baf8-24d185efd1c1,aa8e8c1e-f8a4-4c59-8bf2-a016c6f36b3a
7,Wht conditions for clinical work inclusion in ...,Thank you for your thoughtful question. It’s r...,None,Clinical work can be included in a standard te...,1,1,1,17.681931,fd75f318-755b-4f41-a89f-6d37c6d3dd2a,2543c7c4-d4a7-4e71-8159-424e5b79621d
8,Is the Federal Work-Study program subject to p...,Thank you for your thoughtful question. Based ...,None,The payment period is applicable to all Title ...,1,1,1,4.811328,c5762f9f-6a60-4738-a0aa-45c5892ee6a8,dc03872d-fc98-4728-b48f-676f7eae7d88
9,Chapter 3 include clinical work?,Thank you for your question. Based on the cont...,None,Inclusion of clinical work in a standard term ...,1,1,1,4.113442,f3583fb1-05ab-4b25-9a65-52c7f678a021,0381a317-39f8-46ca-84f4-885a9a69f069


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

The two screenshots are in the directory, experiment-1 and experiment 2

The main difference is empathy. 